In [ ]:
%load_ext autoreload
%autoreload 2

# Level-Set Based Kernel Density Estimation
> Defining the classes `LevelSetKDEx` and `LevelSetKDEx_kNN` which turn any point predictor into a conditional kernel density estimator.

In [ ]:
#| default_exp levelSetKDEx

In [ ]:
#| hide
from nbdev.showdoc import *
# from nbdev.qmd import *

In [ ]:
#| export
from __future__ import annotations
from fastcore.docments import *
from fastcore.test import *
from fastcore.utils import *

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors 
from joblib import Parallel, delayed, dump, load

from dddex.basePredictor import BasePredictor, restructureWeightsDataList

In the following we define the classes `LevelSetKDEx` and `LevelSetKDEx_kNN` where KDE is short for 'Kernel Density Estimator' and the 'x' is supposed to signal that both classes can be defined based on any arbitrary point predictor. The name 'LevelSet' stems from the fact that every approach presented in this notebook interprets the values of the point forecasts as a similarity measure between samples. The point predictor is specified by the argument `estimator` and must have a `.predict()`-method and should have been trained before hand. 

Both classes `LevelSetKDEx` and `LevelSetKDEx_kNN` fulfill the same task: By first running `.fit(XTrain, yTrain)` and then calling `.generateWeights(XTest)`, they both output an estimation of the conditional density of every sample specified by 'XTest'. The basic idea for both approaches is also identical: Suppose we have a single test sample at hand. At first, we compare the value of the point prediction of this sample and the values of the point predictions of the training samples computed via `estimator.predict(XTrain)` and `estimator.predict(XTest)`, respectively. Based on this comparison, we select 'binSize'-many training samples that we deem the most similar to the test sample at hand. The concrete way we select the training samples constitutes the only difference between `LevelSetKDEx` and `LevelSetKDEx_kNN`. Finally, the empirical distribution of the y-values of these training samples then acts as our estimation of the conditional distribution.

Further details on how both approaches work approaches can be found below.

## Level-Set Approach based on Bin Building

In [ ]:
#| export

class LevelSetKDEx(BasePredictor):
    """
    `LevelSetKDEx`
    """
    
    def __init__(self, 
                 estimator, # (Fitted) object with a .predict-method.
                 binSize: int = None # Size of the bins created to group the training samples.
                 ):
        
        if not (hasattr(estimator, 'predict') and callable(estimator.predict)):
            raise ValueError("'estimator' has to have a 'predict'-method!")
        else:
            self.estimator = estimator
            
        if not (isinstance(binSize, (int, np.integer)) or binSize is None):
            raise ValueError("'binSize' has to be integer (or None if it is supposed to be tuned)!")
        else:
            self.binSize = binSize
        
        self.estimator = estimator
        self.binSize = binSize
        
        self.y = None
        self.yPred = None
        self.binPerTrainPred = None
        self.indicesPerBin = None
        self.nearestNeighborsOnPreds = None
        
    #---
    
    def __str__(self):
        return f"LevelSetKDEx(estimator = {self.estimator}, binSize = {self.binSize})"
    __repr__ = __str__      
    
    #---
    
    def fit(self, 
            X: np.ndarray, # Feature matrix used by 'estimator' to predict 'y'.
            y: np.ndarray, # 1-dimensional target variable corresponding to the features 'X'.
            ):

        if self.binSize > y.shape[0]:
            raise ValueError("'binSize' mustn't be bigger than the size of 'y'!")

        yPredTrain = self.estimator.predict(X)

        binPerTrainPred, indicesPerBin = generateBins(binSize = self.binSize,
                                                      yPredTrain = yPredTrain)

        #---

        nn = NearestNeighbors(algorithm = 'kd_tree')
        yPredTrain_reshaped = np.reshape(yPredTrain, newshape = (len(yPredTrain), 1))

        nn.fit(X = yPredTrain_reshaped)

        #---

        self.y = y
        self.yPredTrain = yPredTrain
        self.binPerTrainPred = binPerTrainPred
        self.indicesPerBin = indicesPerBin
        self.nearestNeighborsOnPreds = nn
        
    #---
    
    def getWeights(self, 
                   X: np.ndarray, # Feature matrix of samples for which conditional density estimates are computed.
                   outputType: 'all' | # Specifies structure of output.
                               'onlyPositiveWeights' | 
                               'summarized' | 
                               'cumulativeDistribution' | 
                               'cumulativeDistributionSummarized' = 'onlyPositiveWeights', 
                   scalingList: list | np.ndarray | None = None, # List or array with same size as self.y containing floats being multiplied with self.y.
                   ):
        
        binPerTrainPred = self.binPerTrainPred
        indicesPerBin = self.indicesPerBin
        nearestNeighborsOnPreds = self.nearestNeighborsOnPreds

        #---

        yPred = self.estimator.predict(X)   
        yPred_reshaped = np.reshape(yPred, newshape = (len(yPred), 1))

        nearestPredIndex = nearestNeighborsOnPreds.kneighbors(X = yPred_reshaped, 
                                                              n_neighbors = 1, 
                                                              return_distance = False).ravel()

        nearestPredNeighbor = self.yPredTrain[nearestPredIndex]

        neighborsList = [indicesPerBin[binPerTrainPred[nearestPredNeighbor[i]]] for i in range(len(yPred))]

        #---

        # Checks        
        for i in range(len(neighborsList)):
            if len(neighborsList[i]) < self.binSize:
                ipdb.set_trace()

        #---

        # weightsDataList is a list whose elements correspond to one test prediction each. 
        weightsDataList = [(np.repeat(1 / len(neighbors), len(neighbors)), np.array(neighbors)) for neighbors in neighborsList]

        weightsDataList = restructureWeightsDataList(weightsDataList = weightsDataList, 
                                                     outputType = outputType, 
                                                     y = self.y,
                                                     scalingList = scalingList,
                                                     equalWeights = True)

        return weightsDataList
    

In [ ]:
# show_doc(LevelSetKDEx)

In [ ]:
# show_doc(LevelSetKDEx.fit)

In [ ]:
show_doc(LevelSetKDEx.getWeights)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDE.py#L101){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDE.getWeights

>      LevelSetKDE.getWeights (X:numpy.ndarray, outputType:Union[ForwardRef('all
>                              '),ForwardRef('onlyPositiveWeights'),ForwardRef('
>                              summarized'),ForwardRef('cumulativeDistribution')
>                              ,ForwardRef('cumulativeDistributionSummarized')]=
>                              'onlyPositiveWeights', scalingList:Union[list,num
>                              py.ndarray,NoneType]=None)

Compute weights for every sample specified by feature matrix 'X'

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | np.ndarray |  | Feature matrix of samples for which conditional density estimates are computed. |
| outputType | 'all' \| 'onlyPositiveWeights' \| 'summarized' \| 'cumulativeDistribution' \| 'cumulativeDistributionSummarized' | onlyPositiveWeights | Specifies structure of output. |
| scalingList | list \| np.ndarray \| None | None | List or array with same size as self.y containing floats being multiplied with self.y. |

#### Generate Bins

In [ ]:
#| export

def generateBins(binSize: int, # Size of the bins of values being grouped together.
                 yPredTrain: np.ndarray, # 1-dimensional array of predicted values.
                 ):
    "Used to generate the bin-structure induced by the Level-Set-Forecaster algorithm"
    
    yPredTrainUnique = pd.Series(yPredTrain).unique()
    predIndicesSort = np.argsort(yPredTrainUnique)
    
    yPredTrainUniqueSorted = yPredTrainUnique[predIndicesSort]
    indicesByPredTrain = [np.where(pred == yPredTrain)[0] for pred in yPredTrainUniqueSorted]
    
    currentBinSize = 0
    binIndex = 0
    binExisting = False
    trainIndicesLeft = len(yPredTrain)
    binPerPred = dict()
    indicesPerBin = dict()

    for i in range(len(indicesByPredTrain)):
        currentBinSize += len(indicesByPredTrain[i])
        binPerPred[yPredTrainUniqueSorted[i]] = binIndex
        
        if binExisting:
            indicesPerBin[binIndex] = np.append(indicesPerBin[binIndex], indicesByPredTrain[i])
        else:
            indicesPerBin[binIndex] = indicesByPredTrain[i]
            binExisting = True

        trainIndicesLeft -= len(indicesByPredTrain[i])
        if trainIndicesLeft < binSize:
            for j in np.arange(i+1, len(indicesByPredTrain), 1):
                binPerPred[yPredTrainUniqueSorted[j]] = binIndex
                indicesPerBin[binIndex] = np.append(indicesPerBin[binIndex], indicesByPredTrain[j])
            break

        if currentBinSize >= binSize:
            binIndex += 1
            currentBinSize = 0
            binExisting = False
    
    #---
    
    # Checks
    
    indices = np.array([])
    for k in range(len(indicesPerBin.keys())):
        indices = np.append(indices, indicesPerBin[k])
 
    if len(indices) != len(yPredTrain):
        ipdb.set_trace()
    
    predCheck = np.array([pred in binPerPred.keys() for pred in yPredTrain])
    keyCheck = np.array([key in yPredTrain for key in binPerPred.keys()])
    
    if (all(predCheck) & all(keyCheck)) is False:
        ipdb.set_trace()
    
    return binPerPred, indicesPerBin

## Level-Set Approach based on kNN

In [ ]:
#| export

class LevelSetKDEx_kNN(BasePredictor):
    """
     `LevelSetKDEx_kNN` turns any point predictor that has a .predict-method 
    into an estimator of the condititional density of the underlying distribution.
    The basic idea of each level-set based approach is to interprete the point forecast
    generated by the underlying point predictor as a similarity measure of samples.
    In the case of the `LevelSetKDEx_kNN` defined here, for every new samples
    'binSize'-many training samples are computed whose point forecast is closest
    to the point forecast of the new sample.
    The resulting empirical distribution of these 'nearest' training samples are 
    viewed as our estimation of the conditional distribution of each the new sample 
    at hand.
    
    NOTE 1: The `LevelSetKDEx_kNN` class can only be applied to estimators that 
    have been fitted already.
    
    NOTE 2: In contrast to the standard `LevelSetKDEx`, it is possible to apply
    `LevelSetKDEx_kNN` to arbitrary dimensional point predictors.
    """
    
    def __init__(self, 
                 estimator, # Object with a .predict-method (fitted).
                 binSize: int | None = None, # Size of the neighbors considered to compute conditional density.
                 ):
        
        if not (hasattr(estimator, 'predict') and callable(estimator.predict)):
            raise ValueError("'estimator' has to have a 'predict'-method!")
        else:
            self.estimator = estimator
            
        if not isinstance(binSize, (int, np.integer)):
            raise ValueError("'binSize' has to be integer!")
        else:
            self.binSize = binSize
        
        self.estimator = estimator
        self.binSize = binSize
        
        self.y = None
        self.yPred = None
        self.nearestNeighborsOnPreds = None
        
    #---
    
    def __str__(self):
        return f"LevelSetKDEx_kNN(estimator = {self.estimator}, binSize = {self.binSize})"
    __repr__ = __str__   
    
    #---
    
    def fit(self:LevelSetKDEx_kNN, 
            X: np.ndarray, # Feature matrix used by 'estimator' to predict 'y'.
            y: np.ndarray, # Target variable corresponding to features 'X'.
            ):

        if self.binSize > y.shape[0]:
            raise ValueError("'binSize' mustn't be bigger than the size of 'y'!")

        yPredTrain = self.estimator.predict(X)
        yPredTrain_reshaped = np.reshape(yPredTrain, newshape = (len(yPredTrain), 1))

        nn = NearestNeighbors(algorithm = 'kd_tree')
        nn.fit(X = yPredTrain_reshaped)

        #---

        self.y = y
        self.yPredTrain = yPredTrain
        self.nearestNeighborsOnPreds = nn
        
    #---
    
    def getWeights(self: LevelSetKDEx_kNN, 
                   X: np.ndarray, # Feature matrix of samples for which conditional density estimates are computed.
                   outputType: 'all' | # Specifies structure of output.
                               'onlyPositiveWeights' | 
                               'summarized' | 
                               'cumulativeDistribution' | 
                               'cumulativeDistributionSummarized' = 'onlyPositiveWeights', 
                   scalingList: list | np.ndarray | None = None, # List or array with same size as self.y containing floats being multiplied with self.y.
                   ):

        nn = self.nearestNeighborsOnPreds

        #---

        yPred = self.estimator.predict(X)   
        yPred_reshaped = np.reshape(yPred, newshape = (len(yPred), 1))

        distancesDf, neighborsMatrix = nn.kneighbors(X = yPred_reshaped, 
                                                     n_neighbors = self.binSize + 1)

        #---

        neighborsList = list(neighborsMatrix[:, 0:self.binSize])
        distanceCheck = np.where(distancesDf[:, self.binSize - 1] == distancesDf[:, self.binSize])
        indicesToMod = distanceCheck[0]

        for index in indicesToMod:
            distanceExtremePoint = np.absolute(yPred[index] - self.yPredTrain[neighborsMatrix[index, self.binSize-1]])

            neighborsByRadius = nn.radius_neighbors(X = yPred_reshaped[index:index + 1], 
                                                    radius = distanceExtremePoint, return_distance = False)[0]
            neighborsList[index] = neighborsByRadius

        #---

        for i in range(len(neighborsList)):
            if len(neighborsList[i]) < self.binSize:
                ipdb.set_trace()

        #---

        # weightsDataList is a list whose elements correspond to one test prediction each. 
        weightsDataList = [(np.repeat(1 / len(neighbors), len(neighbors)), np.array(neighbors)) for neighbors in neighborsList]

        weightsDataList = restructureWeightsDataList(weightsDataList = weightsDataList, 
                                                     outputType = outputType, 
                                                     y = self.y,
                                                     scalingList = scalingList,
                                                     equalWeights = True)

        return weightsDataList
      

In [ ]:
# show_doc(LevelSetKDEx_kNN)

In [ ]:
# show_doc(LevelSetKDEx_kNN.fit)

In [ ]:
# show_doc(LevelSetKDEx_kNN.getWeights)

## LSF Cross-Validation

In [ ]:
#| export

class binSizeCV:

    def __init__(self,
                 estimator, # Object with a .predict-method (fitted).
                 cv, # Specifies cross-validation-splits. Identical to 'cv' used for cross-validation in sklearn.
                 LSF_type: 'LSF' | 'LSF_kNN', # Specifies which LSF-Object we work with during cross-validation.
                 binSizeGrid: list | np.ndarray = [4, 7, 10, 15, 20, 30, 40, 50, 60, 70, 80, 
                                                   100, 125, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900,
                                                   1000, 1250, 1500, 1750, 2000, 2500, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000], # binSize (int) values being evaluated.                
                 probs: list | np.ndarray = [i / 100 for i in range(1, 100, 1)], # list or array of floats between 0 and 1. p-quantiles being predicted to evaluate performance of LSF.
                 refitPerProb: bool = False, # If True, for each p-quantile a fitted LSF with best binSize to predict it is returned. Otherwise only one LSF is returned that is best over all probs.
                 n_jobs: int | None = None, # number of folds being computed in parallel.
                 ):
        
        # CHECKS
        
        if isinstance(estimator, (LevelSetKDEx, LevelSetKDEx_kNN)):
            raise ValueError("'estimator' has to be a point predictor and not a LevelSetKDEx-Object!")   
        elif not (hasattr(estimator, 'predict') and callable(estimator.predict)):
            raise ValueError("'estimator' has to have a 'predict'-method!")
        else:
            self.estimator = estimator
            
        if LSF_type is None or not LSF_type in ["LSF", "LSF_kNN"]:
            raise ValueError("LSF_type must be specified and must either be 'LSF' or 'LSF_kNN'!")
        else:
            self.LSF_type = LSF_type
            
        if np.any(np.array(probs) > 1) or np.any(np.array(probs) < 0): 
            raise ValueError("probs must only contain numbers between 0 and 1!")
        else:
            self.probs = probs
        
        #---
        
        self.binSizeGrid = binSizeGrid        
        self.cv = cv
        self.refitPerProb = refitPerProb
        self.n_jobs = n_jobs
        
        self.best_binSize = None
        self.best_binSize_perProb = None
        self.best_estimatorLSF = None
        self.cv_results = None
        self.cv_results_raw = None
        

In [ ]:
# show_doc(binSizeCV)

In [ ]:
#| export

@patch
def fit(self: binSizeCV, 
        X, 
        y):
    
    scoresPerFold = Parallel(n_jobs = self.n_jobs)(delayed(scoresForFold)(cvFold = cvFold,
                                                                          binSizeGrid = self.binSizeGrid,
                                                                          probs = self.probs,
                                                                          estimator = self.estimator,
                                                                          LSF_type = self.LSF_type,
                                                                          y = y,
                                                                          X = X) for cvFold in cvFolds)    

    self.cv_results_raw = scoresPerFold

    #---

    nvCostsMatrix = scoresPerFold[0]

    for i in range(1, len(scoresPerFold)):
        nvCostsMatrix = nvCostsMatrix + scoresPerFold[i]

    nvCostsMatrix = nvCostsMatrix / len(cvFolds)

    self.cv_results = nvCostsMatrix

    #---

    meanCostsDf = nvCostsMatrix.mean(axis = 1)
    binSizeBestOverall = meanCostsDf.index[np.argmax(meanCostsDf)]
    self.best_binSize = binSizeBestOverall

    binSizeBestPerProb = nvCostsMatrix.idxmax(axis = 0)
    self.best_binSize_perProb = binSizeBestPerProb

    #---

    if self.refitPerProb:

        LSFDict = dict()
        for binSize in binSizeBestPerProb.unique():

            if self.LSF_type == 'LSF':
                LSF = LevelSetKDEx(estimator = self.estimator, 
                                         binSize = binSize)
            else:
                LSF = LevelSetKDEx_kNN(estimator = self.estimator, 
                                             binSize = binSize)

            LSF.fit(X = X, y = y)
            LSFDict[binSize] = LSF

        self.best_estimatorLSF = {prob: LSFDict[binSizeBestPerProb.loc[prob]] 
                                  for prob in binSizeBestPerProb.index}

    else:
        if self.LSF_type == 'LSF':
            LSF = LevelSetKDEx(estimator = self.estimator, 
                                     binSize = binSizeBestOverall)
        else:
            LSF = LevelSetKDEx_kNN(estimator = self.estimator, 
                                         binSize = binSizeBestOverall)

        LSF.fit(X = X, y = y)

        self.best_estimatorLSF = LSF

In [ ]:
show_doc(binSizeCV.fit)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDE.py#L385){target="_blank" style="float:right; font-size:smaller"}

### binSizeCV.fit

>      binSizeCV.fit (X, y)

#### Scores for Single Fold

In [ ]:
#| export

# This function evaluates the newsvendor performance for different bin sizes for one specific fold.
# The considered bin sizes

def scoresForFold(cvFold, binSizeGrid, probs, estimator, LSF_type, y, X):
   
    indicesTrain = cvFold[0]
    indicesTest = cvFold[1]
    
    yTrainFold = y[indicesTrain]
    XTrainFold = X[indicesTrain]
    
    yTestFold = y[indicesTest]
    XTestFold = X[indicesTest]
    
    estimator.fit(X = XTrainFold, y = yTrainFold)
    
    #---
       
    SAA_fold = SAA()
    SAA_fold.fit(y = yTrainFold)
    
    # By setting 'X = None', the SAA results are only computed for a single observation (they are independent of X anyway).
    # In order to receive the final dataframe of SAA results, we simply duplicate this single row as many times as needed.
    quantilesDictSAAOneOb = SAA_fold.predict(X = None, probs = probs, outputAsDf = False)
    quantilesDictSAA = {prob: np.repeat(quantile, len(XTestFold)) for prob, quantile in quantilesDictSAAOneOb.items()}
    
    #---
                                                   
    coefPresPerBinSize = list()
    
    binSizeGrid = [binSize for binSize in binSizeGrid if binSize <= len(YTrainFold)]
    
    for binSize in iter(binSizeGrid):
        
        if LSF_type == 'LSF':
            estimatorLSF = LevelSetKDEx(estimator = estimator,
                                              binSize = binSize)
        else:
            estimatorLSF = LevelSetKDEx_kNN(estimator = estimator,
                                                  binSize = binSize)
        
        estimatorLSF.fit(X = XTrainFold,
                         y = yTrainFold)
        
        quantilesDict = estimatorLSF.predictQ(X = XTestFold,
                                              probs = probs,
                                              outputAsDf = False)
        
        #---
        
        # coeffPres = Coefficient of Prescriptiveness
        
        coefPresDict = {prob: [] for prob in probs}
        
        for prob in probs:            
            coefPres = getCoefPres(decisions = quantilesDict[prob], 
                                   decisionsSAA = quantilesDictSAA[prob], 
                                   yTest = yTestFold, 
                                   prob = prob)
            
            coefPresDict[prob].append(coefPres)
    
    #---
    
    coefPresDf = pd.DataFrame(coefPresDict, index = binSizeGrid)
    
    return coefPresDf

##### Coefficient of Prescriptiveness

In [ ]:
#| export

def getCoefPres(decisions, decisionsSAA, yTest, prob):

    # Newsvendor Costs of our model
    cost = np.array([prob * (yTest[i] - decisions[i]) if yTest[i] > decisions[i] 
                     else (1 - prob) * (decisions[i] - yTest[i]) 
                     for i in range(len(yTest))]).sum()
    
    # Newsvendor Costs of SAA
    costSAA = np.array([prob * (yTest[i] - decisionsSAA[i]) if yTest[i] > decisionsSAA[i] 
                        else (1 - prob) * (decisionsSAA[i] - yTest[i]) 
                        for i in range(len(yTest))]).sum()
    
    #---
    
    # We have to capture the special case of costSAA == 0, because then we can't compute the 
    # Coefficient of Prescriptiveness using the actual definition.
    if costSAA > 0:
        coefPres = 1 - cost / costSAA
    else:
        if cost == 0:
            coefPres = 1
        else:
            coefPres = 0
    
    return coefPres

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()